## Optimizadores, Learning Rate Schedulers

En esta sesión vamos a comparar el resultado diferentes optimizadores y learning rates schedulers 

## Leer, normalizar y particionar datos

In [1]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Optimizadores

Vamos a probar diferentes optimizadores:

In [3]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=5
lr=0.001

opt=[]
opt.append(SGD(learning_rate=lr, momentum=0.9))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))

best_acc=0.0
for optim in opt:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) 
    
    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")


Epoch 1/5
375/375 [==============================] - 17s 45ms/step - loss: 1.2061 - accuracy: 0.7404 - val_loss: 0.6052 - val_accuracy: 0.8602
Epoch 2/5
375/375 [==============================] - 15s 41ms/step - loss: 0.4886 - accuracy: 0.8790 - val_loss: 0.4107 - val_accuracy: 0.8931
Epoch 3/5
375/375 [==============================] - 12s 33ms/step - loss: 0.3806 - accuracy: 0.8972 - val_loss: 0.3501 - val_accuracy: 0.9047
Epoch 4/5
375/375 [==============================] - 13s 33ms/step - loss: 0.3352 - accuracy: 0.9065 - val_loss: 0.3163 - val_accuracy: 0.9119
Epoch 5/5
375/375 [==============================] - 12s 33ms/step - loss: 0.3072 - accuracy: 0.9131 - val_loss: 0.2924 - val_accuracy: 0.9191

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Epoch 1/5
375/375 [==============================] - 25s 64ms/step - loss: 0.2096 - accuracy: 0.9360 - val_loss: 0.1098 - val_accuracy: 0.9654
Epoch 2/5
375/375 [==============================] - 23s 61ms/step - loss: 0.0778

## Mejor Optimizador: opt.append(Adam(learning_rate=lr))

## **EJERCICIO**

Añade más epochs a este ejemplo anterior, un early stopping y un model_checkpoint. Luego prueba el test sobre el model checkpoint almacenado.

## Learning Rates Schedulers

Los learning rate schedulers son mecanismos de modificación del learning rate. Normalmente lo que hacen es reducir el valor del learning rate, lo que se conoce como ***learning rate annealing***. Esta modificación se suele realizar al acabar cada epoch.

Keras ya dispone de algunos learning rate schedulers implementados pero el usuario puede implemetar su propia estrategia de annealing. Veamos ambos casos.

### LRS ya implementado en Keras

In [4]:
# Emplear un LRS estandard de Keras: ReduceLROnPlateau

from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr])  ## <--- aquí está



Epoch 1/5
375/375 [==============================] - 26s 66ms/step - loss: 0.3328 - accuracy: 0.9140 - val_loss: 0.1579 - val_accuracy: 0.9515 - lr: 0.0100
Epoch 2/5
375/375 [==============================] - 22s 59ms/step - loss: 0.1608 - accuracy: 0.9538 - val_loss: 0.2119 - val_accuracy: 0.9464 - lr: 0.0100
Epoch 3/5
375/375 [==============================] - 22s 60ms/step - loss: 0.1266 - accuracy: 0.9645 - val_loss: 0.1553 - val_accuracy: 0.9564 - lr: 0.0100
Epoch 4/5
375/375 [==============================] - 22s 59ms/step - loss: 0.1101 - accuracy: 0.9689 - val_loss: 0.2115 - val_accuracy: 0.9501 - lr: 0.0100
Epoch 5/5
375/375 [==============================] - 22s 59ms/step - loss: 0.1102 - accuracy: 0.9705 - val_loss: 0.1482 - val_accuracy: 0.9628 - lr: 0.0100


### LRS propio mediante función 

La función toma como entrada el epoch

In [5]:
## Emplear un LRS propio: LearningRateScheduler

from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

LRS=LearningRateScheduler(scheduler, verbose=1)

model = Sequential()

model.add(Input(784))

model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])   

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])

 
                



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/5
375/375 [==============================] - 23s 59ms/step - loss: 0.2129 - accuracy: 0.9366 - val_loss: 0.1180 - val_accuracy: 0.9626 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/5
375/375 [==============================] - 22s 59ms/step - loss: 0.0778 - accuracy: 0.9763 - val_loss: 0.0936 - val_accuracy: 0.9714 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/5
375/375 [==============================] - 22s 58ms/step - loss: 0.0500 - accuracy: 0.9832 - val_loss: 0.0779 - val_accuracy: 0.9784 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/5
375/375 [==============================] - 24s 63ms/step - loss: 0.0347 - accuracy: 0.9887 - val_loss: 0.0905 - val_accuracy: 0.9747 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/5
375/375 [==============================] - 20s

### LRS propio tipo CosineAnnealing

![cosine annealing](cosine.png)


In [6]:
# Emplear un LRS propio, CosineAnnealingScheduler

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.001
lr_min=0.00001
epochs=100
def cosine_annealing(x): # recordemos, x es el número de epoch
    lr = lr_max/2 * (1 + math.cos(math.pi * x / epochs))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing)

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])



Epoch 1/100
375/375 [==============================] - 17s 44ms/step - loss: 0.2130 - accuracy: 0.9358 - val_loss: 0.1295 - val_accuracy: 0.9611 - lr: 0.0010
Epoch 2/100
266/375 [====================>.........] - ETA: 4s - loss: 0.0770 - accuracy: 0.9766

KeyboardInterrupt: 

### LRS propio tipo CosineAnnealing with restarts

![cosine annealing restarts](cosinerestart.png)

Este scheduler tiene sentido cuanto queremos guardarnos cada uno de los estado del modelo alcanzado en el mínimo LR para luego combinarlos.

In [ ]:
# Emplear un LRS propio, CosineAnnealingScheduler with restarts

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.001
lr_min=0.00001
epochs=100

def cosine_annealing_with_restarts(x):
    lr = lr_max/2 * (1 + math.cos(math.pi * (x % (epochs/5)) / (epochs/5)))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing_with_restarts)

model = Sequential()
model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])

## Ejercicio

Prueba los diferentes LRS sobre el modelo anterior. Emplea la que creas que es la mejor topología de red y optimizador. Emplea early stopping y model checkpoint. Calcula el accuracy sobre el test con el modelo almacenado en el model checkpoint.



In [ ]:
## Solución




---

## EXTRA, estandaricemos la clase MLP

Vamos a implementar una clase para ahorrarnos escribir código recurrentemente. 
Para ello empleamos la **functional api** de Keras:

In [ ]:
from tensorflow import keras
from keras.layers import Dense

class MLP(keras.Model):

  def __init__(self, input_size,num_classes,hidden=[128]):
    super().__init__()
    
    self.hidden = []
    self.num_h=len(hidden)

    for h in hidden:
       self.hidden.append(Dense(h, activation='relu'))
    self.out = Dense(num_classes, activation='softmax')
    

  def call(self, inputs, training=False):
    
    x=self.hidden[0](inputs)
    for h in range(1,self.num_h):
        x = self.hidden[h](x)
    x = self.out(x)
    return x
    


model = MLP(784,10,[1024,512])  ## <-- aquí se instancia el modelo, input, num clases y lista con hidden layers
model.build((None,784)) ## Esto es necesario para poder instaciar adecuadamente todos los shapes del grafo de computación


model.summary()